# Retrieval Augmentation

In [1]:
def format_text(text):
    return text.strip().replace(". ", ".\n")

In [7]:
from datasets import load_dataset

corpus = load_dataset(path="wikipedia", name="20220301.simple", split="train[:10000]", )

Found cached dataset wikipedia (C:/Users/matte/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [ ]:
load_dataset()